In [2]:
from transformers import DecisionTransformerModel, DecisionTransformerConfig
import numpy as np
import torch

device = "cpu"

scale = 1000.0  # normalization for rewards/returns
TARGET_RETURN = 3600 / scale  # evaluation conditioning targets, 3600 is reasonable from the paper LINK
state_dim = (3, 32, 32)
state_dim_flatten = state_dim[0]*state_dim[1]*state_dim[2]
act_dim = 6

/home/pedron18/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
t = 3

target_return = torch.tensor(TARGET_RETURN, device=device, dtype=torch.float32).reshape(1, 1)
states = torch.rand(state_dim, device=device, dtype=torch.float32).reshape(1, 1, state_dim_flatten)
actions = torch.rand((1, act_dim), device=device, dtype=torch.float32)
rewards = torch.rand((1, 1), device=device, dtype=torch.float32)
timesteps = torch.tensor(1, device=device, dtype=torch.long).reshape(1, 1)
attention_mask = torch.zeros(1, 1, device=device, dtype=torch.float32)

print("target_return",target_return.shape)
print("states",states.shape)
print("actions",actions.shape)
print("rewards",rewards.shape)
print("timesteps", target_return.shape)
print("attention_mask", attention_mask.shape)

target_return torch.Size([1, 1])
states torch.Size([1, 1, 3072])
actions torch.Size([1, 6])
rewards torch.Size([1, 1])
timesteps torch.Size([1, 1])
attention_mask torch.Size([1, 1])


In [19]:
t = 3 # number of elements in sequence
b = 2 # batch 

target_return = torch.rand((b, 1), device=device, dtype=torch.float32) # expected future return

states = torch.rand((b, t, state_dim_flatten), device=device, dtype=torch.float32)#.reshape(1, 1, state_dim_flatten) #
actions = torch.rand((b, t, act_dim), device=device, dtype=torch.float32)

rewards = torch.rand((b, t), device=device, dtype=torch.float32)
timesteps = torch.randint_like(torch.zeros((b, t)), device=device, low = 0, high = 50, dtype=torch.long) # integer what timestep we on

attention_mask = torch.zeros((b, t), device=device, dtype=torch.float32)

print("target_return",target_return.shape)
print("states",states.shape)
print("actions",actions.shape)
print("rewards",rewards.shape)
print("timesteps", timesteps.shape)
print("attention_mask", attention_mask.shape)

target_return torch.Size([2, 1])
states torch.Size([2, 3, 3072])
actions torch.Size([2, 3, 6])
rewards torch.Size([2, 3])
timesteps torch.Size([2, 3])
attention_mask torch.Size([2, 3])


In [20]:
config = DecisionTransformerConfig(state_dim_flatten, act_dim, max_ep_len=4096)


model = DecisionTransformerModel(config).to(device)
model.eval()

state_preds, action_preds, return_preds = model(states=states,
        actions=actions,
        rewards=rewards,
        returns_to_go=target_return,
        timesteps=timesteps,
        attention_mask=attention_mask,
        return_dict=False)

#print("argmax", torch.argmax(action_preds).item())
print("state_preds",state_preds.shape)
print("action_preds",action_preds.shape)
print("return_preds", return_preds.shape)

state_preds, action_preds, return_preds

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

In [ ]:
import random

population = [0, 1, 2, 3, 4]
ls = random.sample(population,5)

print("Pure random: ", ls)

max_sequence = 4

def sample_with_order(population, max_sequence_length):
    """
        Samples [1,population] elements from a sequence mantaining the order
    """
    k = random.randint(1, max_sequence_length)
    k_start = random.randint(0, len(population)-k)
    return population[k_start: k_start+k]


print(sample_with_order(population, max_sequence))

   

Pure random:  [0, 1, 4, 2, 3]
[1, 2, 3, 4]


In [ ]:
target_return = torch.ones(1, device=device, dtype=torch.float32).reshape(100, 1)

RuntimeError: shape '[100, 1]' is invalid for input of size 1